# PM Tiles demo

Denne notebooken er ment å vise frem kapabilitetene i PM Tiles formatet.

## Systemkrav

Demoen krever noen verktøy som ikke kan håndteres isolert i et python miljø,
for at dette skal være så portabelt som mulig er det lagt opp til å bruke Docker
til å håndtere miljøet for disse verktøyene.

Blokken under sjekker at Docker og Docker-compose er installert på vertssystemet.

In [2]:
import subprocess
from typing import List

def execute_command(cmd: List[str], **kwargs):
    return subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        **kwargs
    )

system_requirements = [
    ["docker-compose", "--version"],
    ["docker", "--version"],
]

for requirement in system_requirements:
    assert execute_command(requirement, check=False).returncode == 0, f"System requirement {requirement} is not satisfied"

## Input data



### Raster


![nedlasting](../../public/img/geonorge-n50-raster-krs.png)

Last ned filen som vist over, pakk ut og flytt .tif filen til `raster/data/32_N50raster_2024.tif`


### Vektor 

![nedlasting](../../public/img/geonorge-n50-vektor-agder.png)

Last ned filen som vist over, pakk ut og flytt .gdb filen til `vector/data/Basisdata_42_Agder_25832_N50Kartdata_FGDB.gdb`

Først sjekker vi at input filene finnes som forventet.

In [3]:
import utils

assert utils.path_exists("raster/data/32_N50raster_2024.tif"), f"Mangler inputfil raster/data/32_N50raster_2024.tif"
assert utils.path_exists("vector/data/Basisdata_42_Agder_25832_N50Kartdata_FGDB.gdb"), f"Mangler inputfil vector/data/Basisdata_42_Agder_25832_N50Kartdata_FGDB.gdb"

## Konverterer raster data

Merk at her brukes python komandolinjeverktøyet `rio-pmtiles` som installeres via requirements.txt filen.

```bash
# Etter at virtuelt python miljø er aktivert.
pip install -r requirements.txt
```

In [4]:
# Oppretter mappe for output
utils.create_dir("raster/out")


Directory 'raster/out' created successfully.


In [5]:
# Genererer pmtiles
!rio pmtiles raster/data/32_N50raster_2024.tif raster/out/N50_raster_2024.pmtiles --format PNG


100%|██████████| 1425/1425 [00:12<00:00, 110.42it/s]


## Konverterer vektor data

In [6]:
# Oppretter mappe for output
utils.create_dir("vector/out")

Directory 'vector/out' created successfully.


### Dockerfil

For at demoen skal være relativt portabel skjer selve transformasjonen fra gdb til pmtiles
i en docker container gjennom en rekke transformasjoner ved hjelp av flere verktøy.

Format transformasjon:

gdb => geojson => mbtiles => pmtiles

Verktøy:

- ogr2ogr: Konverterer gdb til geojson (gdal)
- tippecanoe: Konverterer geojson til mbtiles
- pmtiles: Konverterer mbtiles til pmtiles


Det er mulig å kombinere flere lag i samme .pmtiles fil, men ved bruk av `vector/build.sh` scriptet fungerte
ikke dette helt som forventet. Lagene blir opprettet, men ved lavere zoom ser ikke kartet pent ut. Videreutvikling
oppfordres.

In [7]:
# Bygger docker image med gdal, tippecanoe og pmtiles
!docker build -t pmtiles -f vector/Dockerfile .

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 989B 0.0s done
#1 DONE 0.1s

#2 [internal] load metadata for docker.io/library/ubuntu:latest
#2 ...

#3 [auth] library/ubuntu:pull token for registry-1.docker.io
#3 DONE 0.0s

#2 [internal] load metadata for docker.io/library/ubuntu:latest
#2 DONE 2.9s

#4 [internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [ 1/19] FROM docker.io/library/ubuntu:latest@sha256:9cbed754112939e914291337b5e554b07ad7c392491dba6daf25eef1332a22e8
#5 resolve docker.io/library/ubuntu:latest@sha256:9cbed754112939e914291337b5e554b07ad7c392491dba6daf25eef1332a22e8 0.0s done
#5 DONE 0.1s

#6 [internal] load build context
#6 transferring context: 2.95kB 0.0s done
#6 DONE 0.1s

#5 [ 1/19] FROM docker.io/library/ubuntu:latest@sha256:9cbed754112939e914291337b5e554b07ad7c392491dba6daf25eef1332a22e8
#5 sha256:76249c7cd50397d2e8c06a75106723d057deaba0ffbc7

Starter en container for å kunne kopiere lokale filer over til containeren og kjøre konverteringsscriptet
i det isolerte miljøet.

In [8]:
!docker run --name pmtiles-converter -d pmtiles

2d75bc6df7468810a8bbdfa374f6e02e685e8870f69d83fdac0826059d53d310


Konverterer line endings i shell script fra Windows (CRLF) til Unix (LF) format
for å unngå problemer i Linux containeren.

In [14]:
# Konverter Windows line endings (CRLF) til Unix line endings (LF)
with open('vector/gdb_to_pmtiles.sh', 'rb') as f:
    content = f.read()

# Erstatt \r\n med \n
unix_content = content.replace(b'\r\n', b'\n')

# Skriv tilbake med Unix line endings
with open('vector/gdb_to_pmtiles_unix.sh', 'wb') as f:
    f.write(unix_content)

Kopierer kildedata inn i containeren.

In [9]:
!docker cp ./vector/data/Basisdata_42_Agder_25832_N50Kartdata_FGDB.gdb pmtiles-converter:/app/N50_vektor_agder.gdb 

In [15]:
# Kopierer det konverterte scriptet (med Unix line endings) inn i containeren
!docker cp ./vector/gdb_to_pmtiles_unix.sh pmtiles-converter:/app/gdb_to_pmtiles.sh
!docker exec pmtiles-converter chmod +x /app/gdb_to_pmtiles.sh

Konverterer samferdsel senterlinje laget til pmtiles

In [16]:
!docker exec pmtiles-converter bash gdb_to_pmtiles.sh N50_vektor_agder.gdb N50_Samferdsel_senterlinje N50_Samferdsel_senterlinje.pmtiles

🧩 Converting GDB to GeoJSON...
🗺️ Creating MBTiles with tippecanoe...
📦 Converting MBTiles to PMTiles...
2025/09/09 14:02:42 convert.go:159: Pass 1: Assembling TileID set
2025/09/09 14:02:42 convert.go:190: Pass 2: writing tiles
2025/09/09 14:02:42 convert.go:244: # of addressed tiles:  302
2025/09/09 14:02:42 convert.go:245: # of tile entries (after RLE):  302
2025/09/09 14:02:42 convert.go:246: # of tile contents:  302
2025/09/09 14:02:42 convert.go:269: Total dir bytes:  909
2025/09/09 14:02:42 convert.go:270: Average bytes per addressed tile: 3.01
2025/09/09 14:02:42 convert.go:239: Finished in  35.252657ms
✅ Done! PMTiles file created at: N50_Samferdsel_senterlinje.pmtiles


Read 0.00 million features
Read 0.01 million features
Read 0.02 million features
Read 0.03 million features
Read 0.04 million features
Read 0.05 million features
Read 0.06 million features
Read 0.07 million features
Read 0.08 million features
                              
Reordering geometry: 0% 
Reordering geometry: 1% 
Reordering geometry: 2% 
Reordering geometry: 3% 
Reordering geometry: 4% 
Reordering geometry: 5% 
Reordering geometry: 6% 
Reordering geometry: 7% 
Reordering geometry: 8% 
Reordering geometry: 9% 
Reordering geometry: 10% 
Reordering geometry: 11% 
Reordering geometry: 12% 
Reordering geometry: 13% 
Reordering geometry: 14% 
Reordering geometry: 15% 
Reordering geometry: 16% 
Reordering geometry: 17% 
Reordering geometry: 18% 
Reordering geometry: 19% 
Reordering geometry: 20% 
Reordering geometry: 21% 
Reordering geometry: 22% 
Reordering geometry: 23% 
Reordering geometry: 24% 
Reordering geometry: 25% 
Reordering geometry: 26% 
Reordering geometry: 27% 
Reorderi

Kopierer pmtiles filen ut av containeren

In [17]:
!docker cp pmtiles-converter:/app/N50_Samferdsel_senterlinje.pmtiles ./vector/out/N50_Samferdsel_senterlinje.pmtiles

Konverterer bygninger og anlegg område laget til pmtiles

In [18]:
!docker exec pmtiles-converter bash gdb_to_pmtiles.sh N50_vektor_agder.gdb N50_BygningerOgAnlegg_omrade N50_BygningerOgAnlegg_omrade.pmtiles

🧩 Converting GDB to GeoJSON...
🗺️ Creating MBTiles with tippecanoe...
📦 Converting MBTiles to PMTiles...
2025/09/09 14:03:48 convert.go:159: Pass 1: Assembling TileID set
2025/09/09 14:03:48 convert.go:190: Pass 2: writing tiles
2025/09/09 14:03:48 convert.go:244: # of addressed tiles:  200
2025/09/09 14:03:48 convert.go:245: # of tile entries (after RLE):  200
2025/09/09 14:03:48 convert.go:246: # of tile contents:  200
2025/09/09 14:03:48 convert.go:269: Total dir bytes:  542
2025/09/09 14:03:48 convert.go:270: Average bytes per addressed tile: 2.71
2025/09/09 14:03:48 convert.go:239: Finished in  6.389547ms
✅ Done! PMTiles file created at: N50_BygningerOgAnlegg_omrade.pmtiles


Read 0.00 million features
                              
Reordering geometry: 0% 
Reordering geometry: 1% 
Reordering geometry: 2% 
Reordering geometry: 3% 
Reordering geometry: 4% 
Reordering geometry: 5% 
Reordering geometry: 6% 
Reordering geometry: 7% 
Reordering geometry: 8% 
Reordering geometry: 9% 
Reordering geometry: 10% 
Reordering geometry: 11% 
Reordering geometry: 12% 
Reordering geometry: 13% 
Reordering geometry: 14% 
Reordering geometry: 15% 
Reordering geometry: 16% 
Reordering geometry: 17% 
Reordering geometry: 18% 
Reordering geometry: 19% 
Reordering geometry: 20% 
Reordering geometry: 21% 
Reordering geometry: 22% 
Reordering geometry: 23% 
Reordering geometry: 24% 
Reordering geometry: 25% 
Reordering geometry: 26% 
Reordering geometry: 27% 
Reordering geometry: 28% 
Reordering geometry: 29% 
Reordering geometry: 30% 
Reordering geometry: 31% 
Reordering geometry: 32% 
Reordering geometry: 33% 
Reordering geometry: 34% 
Reordering geometry: 35% 
Reordering geome

Kopierer pmtiles filen ut av containeren

In [19]:
!docker cp pmtiles-converter:/app/N50_BygningerOgAnlegg_omrade.pmtiles ./vector/out/N50_BygningerOgAnlegg_omrade.pmtiles

Stopper og fjerner containeren etter at konverteringen er ferdig.

In [20]:
!docker stop pmtiles-converter
!docker rm pmtiles-converter

pmtiles-converter
pmtiles-converter
pmtiles-converter


Fjerner docker image

In [21]:
!docker rmi pmtiles

Untagged: pmtiles:latest
Deleted: sha256:3806cbbfe373384aef256a51b3a6f7af4632b84f90511d02112c3ac846604947


# Demo

I denne delen bruker vi `docker-compose` til å kjøre en lokal demo hvor pmtiles filene hostes
over nettet og en veldig enkel web-app leser data fra filene.

Demoen består av to applikasjoner:

- Web appen som kjører i nettleseren.
- En ngix container som hoster de tre pmtiles filene vi har generert.

Mens docker compose blokken under kjører kan web appen åpnes på 
[http://localhost:3000](http://localhost:3000).
I appen vil man se de tre filene vi har generert som lag, det kreves ingen back-end prosessering utover
å tilgjengeliggjøre filene over internett. Kildekoden for web appen finnes under `client/` mappen.
Ved å inspisere nettverkstrafikken i nettleseren kan man observere 
hvordan range headeren brukes til å spørre ut deler av filene.

(Demo blokken må stoppes manuelt)

In [22]:
!docker compose up --build

^C


#1 [internal] load local bake definitions
#1 reading from stdin 1.09kB done
#1 DONE 0.0s

#2 [backend internal] load build definition from Dockerfile
#2 transferring dockerfile: 1.07kB done
#2 DONE 0.0s

#3 [frontend internal] load build definition from Dockerfile
#3 transferring dockerfile: 477B done
#3 DONE 0.0s

#4 [frontend internal] load metadata for docker.io/library/nginx:stable-alpine
#4 ...

#5 [auth] library/nginx:pull token for registry-1.docker.io
#5 DONE 0.0s

#6 [auth] library/node:pull token for registry-1.docker.io
#6 DONE 0.0s

#7 [frontend internal] load metadata for docker.io/library/node:18
#7 DONE 2.0s

#8 [backend internal] load metadata for docker.io/library/nginx:latest
#8 DONE 2.1s

#4 [frontend internal] load metadata for docker.io/library/nginx:stable-alpine
#4 DONE 2.1s

#9 [backend internal] load .dockerignore
#9 transferring context: 2B done
#9 DONE 0.0s

#10 [frontend internal] load .dockerignore
#10 transferring context: 2B done
#10 DONE 0.1s

#11 [front

 pmtiles-backend  Built
 pmtiles-frontend  Built
 Network pmtiles_default  Creating
 Network pmtiles_default  Created
 Container pmtiles-frontend-1  Creating
 Container pmtiles-backend-1  Creating
 Container pmtiles-frontend-1  Created
 Container pmtiles-backend-1  Created
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: using the "epoll" event method
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: nginx/1.28.0
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: built by gcc 14.2.0 (Alpine 14.2.0) 
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: OS: Linux 6.6.87.2-microsoft-standard-WSL2
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: getrlimit(RLIMIT_NOFILE): 1048576:1048576
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: start worker processes
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: start worker process 30
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: start worker process 31
frontend-1  | 2025/09/09 14:07:12 [notice] 1#1: start worker process 32
frontend-1  | 2025/09/09 14:07:12

Fjerner demo containere og images

In [23]:
!docker compose rm -f -s -v

Going to remove pmtiles-backend-1, pmtiles-frontend-1


 Container pmtiles-backend-1  Stopping
 Container pmtiles-frontend-1  Stopping
 Container pmtiles-backend-1  Stopped
 Container pmtiles-frontend-1  Stopped
 Container pmtiles-frontend-1  Removing
 Container pmtiles-backend-1  Removing
 Container pmtiles-frontend-1  Removed
 Container pmtiles-backend-1  Removed
